In [1]:
import numpy as np
from obspy.core import UTCDateTime
from obspy import read
from obspy.taup import TauPyModel
from obspy.geodetics import locations2degrees
from glob import glob
import os
import os.path

In [3]:
project_name = 'july6'
GF_list = 'rc_gflist.gflist' # Station file used in fq gen code
vel_mod = 'mojave'
stas_name = GF_list.split('.')[0]
fq_dir = '/hdd/rc_fq/summer23/' + project_name + '/'

ruptures = np.load('/hdd/rc_fq/summer23/' + project_name + '_ruptures.npy')
test_ruptures = ['june30.000000', 'june30.000001', 'june30.000002']

# Where to save arrival times

arrival_save_dir = '/hdd/rc_fq/summer23/' + project_name + '_fq_parrivals/'
if os.path.isdir(arrival_save_dir):
    pass
else:
    os.makedirs(arrival_save_dir)

In [ ]:
# w = open('/home/sdybing/rc_fq/test_p_arrivals.csv', 'w') # saves arrivals for every event in one csv. Not using currently
counter = 0

for rupture in ruptures:
    
    counter += 1
    w = open(arrival_save_dir + stas_name + '_arrival_times_' + rupture + '.csv', 'w') # saves arrivals for each event in a separate csv
    
    print('Rupture ' + rupture + ' (' + str(counter) + '/' + str(len(ruptures)) + ')')
    
    log = glob(fq_dir + 'output/ruptures/' + rupture + '.log')
    f = open(log[0],'r')
    line = f.readlines()
    
    # getting hypocenter location
    hyp_loc_junk = line[16]
    hyp_loc_1 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[0])
    hyp_loc_2 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[1])
    hyp_loc_3 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[2])
    
    epicenter = []
    epicenter.append(hyp_loc_1)
    epicenter.append(hyp_loc_2)
    epicenter.append(hyp_loc_3)
#     print(epicenter)
    
    epi_lon = epicenter[0]
    epi_lat = epicenter[1]
    epi_depth = epicenter[2]
    
    # getting hypocenter time
    hyp_time_junk = line[17]
    hyp_time = hyp_time_junk.split(' ')[2].split('Z')[0]
    time_epi = UTCDateTime(hyp_time)  
    stations = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_names.npy')
    Nsta = len(stations) # number of stations
    
    rootpath = fq_dir + '/output/waveforms/' + rupture + '/' # waveform file locations
    lonlat_load = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_namesandlocs.npy') 
    lonlat = lonlat_load[:,0:2] # station locations
    
    station_catalogue_load = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_namesandlocs.npy')
    station_catalogue = station_catalogue_load[:,2]
    
    velmod = TauPyModel(model = fq_dir + '/structure/' + vel_mod + '.npz') # velocity model
    
    predictedP = 9999 * np.ones(len(stations))
    
    # Get predicted arrivals
    
    for ksta in range(len(stations)):
        
        # Find station coordinates
        i = np.where(station_catalogue == stations[ksta])[0]
        lon_sta = float(lonlat[i,0])
        lat_sta = float(lonlat[i,1])
                        
        deg = locations2degrees(lat_sta, lon_sta, epi_lat, epi_lon) # calculates distance between station loc and epicenter loc
        arrivals = velmod.get_travel_times(source_depth_in_km = epi_depth, distance_in_degree = deg, phase_list = ['P','Pn','p'])
        
        # Determine P and S arrivals
        for kphase in range(len(arrivals)):
            if 'P' == arrivals[kphase].name or 'p' == arrivals[kphase].name or 'Pn' == arrivals[kphase].name:
                if arrivals[kphase].time < predictedP[ksta]:
                    predictedP[ksta] = arrivals[kphase].time
                    
#         print('Station ' + stations[ksta] + ': predicted arrival = ' + str(time_epi + predictedP[ksta]))
        
        line = '%s\t%s\t%s\n'%(rupture, str(stations[ksta]), str(time_epi + predictedP[ksta]))
        # print(line)
        w.write(line)
    
    w.close()

# w.close()



Rupture july6.000000 (1/3300)
Rupture july6.000001 (2/3300)
Rupture july6.000002 (3/3300)
Rupture july6.000003 (4/3300)
Rupture july6.000004 (5/3300)
Rupture july6.000005 (6/3300)
Rupture july6.000006 (7/3300)
Rupture july6.000007 (8/3300)
Rupture july6.000008 (9/3300)
Rupture july6.000009 (10/3300)
Rupture july6.000010 (11/3300)
Rupture july6.000011 (12/3300)
Rupture july6.000012 (13/3300)
Rupture july6.000013 (14/3300)
Rupture july6.000014 (15/3300)
Rupture july6.000015 (16/3300)
Rupture july6.000016 (17/3300)
Rupture july6.000017 (18/3300)
Rupture july6.000018 (19/3300)
Rupture july6.000019 (20/3300)
Rupture july6.000020 (21/3300)
Rupture july6.000021 (22/3300)
Rupture july6.000022 (23/3300)
Rupture july6.000023 (24/3300)
Rupture july6.000024 (25/3300)
Rupture july6.000025 (26/3300)
Rupture july6.000026 (27/3300)
Rupture july6.000027 (28/3300)
Rupture july6.000028 (29/3300)
Rupture july6.000029 (30/3300)
Rupture july6.000030 (31/3300)
Rupture july6.000031 (32/3300)
Rupture july6.000

Rupture july6.000260 (261/3300)
Rupture july6.000261 (262/3300)
Rupture july6.000262 (263/3300)
Rupture july6.000263 (264/3300)
Rupture july6.000264 (265/3300)
Rupture july6.000265 (266/3300)
Rupture july6.000266 (267/3300)
Rupture july6.000267 (268/3300)
Rupture july6.000268 (269/3300)
Rupture july6.000269 (270/3300)
Rupture july6.000270 (271/3300)
Rupture july6.000271 (272/3300)
Rupture july6.000272 (273/3300)
Rupture july6.000273 (274/3300)
Rupture july6.000274 (275/3300)
Rupture july6.000275 (276/3300)
Rupture july6.000276 (277/3300)
Rupture july6.000277 (278/3300)
Rupture july6.000278 (279/3300)
Rupture july6.000279 (280/3300)
Rupture july6.000280 (281/3300)
Rupture july6.000281 (282/3300)
Rupture july6.000282 (283/3300)
Rupture july6.000283 (284/3300)
Rupture july6.000284 (285/3300)
Rupture july6.000285 (286/3300)
Rupture july6.000286 (287/3300)
Rupture july6.000287 (288/3300)
Rupture july6.000288 (289/3300)
Rupture july6.000289 (290/3300)
Rupture july6.000290 (291/3300)
Rupture 

Rupture july6.000519 (520/3300)
Rupture july6.000520 (521/3300)
Rupture july6.000521 (522/3300)
Rupture july6.000522 (523/3300)
Rupture july6.000523 (524/3300)
Rupture july6.000524 (525/3300)
Rupture july6.000525 (526/3300)
Rupture july6.000526 (527/3300)
Rupture july6.000527 (528/3300)
Rupture july6.000528 (529/3300)
Rupture july6.000529 (530/3300)
Rupture july6.000530 (531/3300)
Rupture july6.000531 (532/3300)
Rupture july6.000532 (533/3300)
Rupture july6.000533 (534/3300)
Rupture july6.000534 (535/3300)
Rupture july6.000535 (536/3300)
Rupture july6.000536 (537/3300)
Rupture july6.000537 (538/3300)
Rupture july6.000538 (539/3300)
Rupture july6.000539 (540/3300)
Rupture july6.000540 (541/3300)
Rupture july6.000541 (542/3300)
Rupture july6.000542 (543/3300)
Rupture july6.000543 (544/3300)
Rupture july6.000544 (545/3300)
Rupture july6.000545 (546/3300)
Rupture july6.000546 (547/3300)
Rupture july6.000547 (548/3300)
Rupture july6.000548 (549/3300)
Rupture july6.000549 (550/3300)
Rupture 

Rupture july6.000776 (777/3300)
Rupture july6.000777 (778/3300)
Rupture july6.000778 (779/3300)
Rupture july6.000779 (780/3300)
Rupture july6.000780 (781/3300)
Rupture july6.000781 (782/3300)
Rupture july6.000782 (783/3300)
Rupture july6.000783 (784/3300)
Rupture july6.000784 (785/3300)
Rupture july6.000785 (786/3300)
Rupture july6.000786 (787/3300)
Rupture july6.000787 (788/3300)
Rupture july6.000788 (789/3300)
Rupture july6.000789 (790/3300)
Rupture july6.000790 (791/3300)
Rupture july6.000791 (792/3300)
Rupture july6.000792 (793/3300)
Rupture july6.000793 (794/3300)
Rupture july6.000794 (795/3300)
Rupture july6.000795 (796/3300)
Rupture july6.000796 (797/3300)
Rupture july6.000797 (798/3300)
Rupture july6.000798 (799/3300)
Rupture july6.000799 (800/3300)
Rupture july6.000800 (801/3300)
Rupture july6.000801 (802/3300)
Rupture july6.000802 (803/3300)
Rupture july6.000803 (804/3300)
Rupture july6.000804 (805/3300)
Rupture july6.000805 (806/3300)
Rupture july6.000806 (807/3300)
Rupture 

Rupture july6.001032 (1033/3300)
Rupture july6.001033 (1034/3300)
Rupture july6.001034 (1035/3300)
Rupture july6.001035 (1036/3300)
Rupture july6.001036 (1037/3300)
Rupture july6.001037 (1038/3300)
Rupture july6.001038 (1039/3300)
Rupture july6.001039 (1040/3300)
Rupture july6.001040 (1041/3300)
Rupture july6.001041 (1042/3300)
Rupture july6.001042 (1043/3300)
Rupture july6.001043 (1044/3300)
Rupture july6.001044 (1045/3300)
Rupture july6.001045 (1046/3300)
Rupture july6.001046 (1047/3300)
Rupture july6.001047 (1048/3300)
Rupture july6.001048 (1049/3300)
Rupture july6.001049 (1050/3300)
Rupture july6.001050 (1051/3300)
Rupture july6.001051 (1052/3300)
Rupture july6.001052 (1053/3300)
Rupture july6.001053 (1054/3300)
Rupture july6.001054 (1055/3300)
Rupture july6.001055 (1056/3300)
Rupture july6.001056 (1057/3300)
Rupture july6.001057 (1058/3300)
Rupture july6.001058 (1059/3300)
Rupture july6.001059 (1060/3300)
Rupture july6.001060 (1061/3300)
Rupture july6.001061 (1062/3300)
Rupture ju

Rupture july6.001281 (1282/3300)
Rupture july6.001282 (1283/3300)
Rupture july6.001283 (1284/3300)
Rupture july6.001284 (1285/3300)
Rupture july6.001285 (1286/3300)
Rupture july6.001286 (1287/3300)
Rupture july6.001287 (1288/3300)
Rupture july6.001288 (1289/3300)
Rupture july6.001289 (1290/3300)
Rupture july6.001290 (1291/3300)
Rupture july6.001291 (1292/3300)
Rupture july6.001292 (1293/3300)
Rupture july6.001293 (1294/3300)
Rupture july6.001294 (1295/3300)
Rupture july6.001295 (1296/3300)
Rupture july6.001296 (1297/3300)
Rupture july6.001297 (1298/3300)
Rupture july6.001298 (1299/3300)
Rupture july6.001299 (1300/3300)
Rupture july6.001300 (1301/3300)
Rupture july6.001301 (1302/3300)
Rupture july6.001302 (1303/3300)
Rupture july6.001303 (1304/3300)
Rupture july6.001304 (1305/3300)
Rupture july6.001305 (1306/3300)
Rupture july6.001306 (1307/3300)
Rupture july6.001307 (1308/3300)
Rupture july6.001308 (1309/3300)
Rupture july6.001309 (1310/3300)
Rupture july6.001310 (1311/3300)
Rupture ju

Rupture july6.001530 (1531/3300)
Rupture july6.001531 (1532/3300)
Rupture july6.001532 (1533/3300)
Rupture july6.001533 (1534/3300)
Rupture july6.001534 (1535/3300)
Rupture july6.001535 (1536/3300)
Rupture july6.001536 (1537/3300)
Rupture july6.001537 (1538/3300)
Rupture july6.001538 (1539/3300)
Rupture july6.001539 (1540/3300)
Rupture july6.001540 (1541/3300)
Rupture july6.001541 (1542/3300)
Rupture july6.001542 (1543/3300)
Rupture july6.001543 (1544/3300)
Rupture july6.001544 (1545/3300)
Rupture july6.001545 (1546/3300)
Rupture july6.001546 (1547/3300)
Rupture july6.001547 (1548/3300)
Rupture july6.001548 (1549/3300)
Rupture july6.001549 (1550/3300)
Rupture july6.001550 (1551/3300)
Rupture july6.001551 (1552/3300)
Rupture july6.001552 (1553/3300)
Rupture july6.001553 (1554/3300)
Rupture july6.001554 (1555/3300)
Rupture july6.001555 (1556/3300)
Rupture july6.001556 (1557/3300)
Rupture july6.001557 (1558/3300)
Rupture july6.001558 (1559/3300)
Rupture july6.001559 (1560/3300)
Rupture ju